# Hearthstone Project
***
***

# Goals
- Thoroughly prepare so that it is ready for exploration and modeling
    - I won't be creating any models for this project but I'd still like to prepare the data nonetheless
- Explore the data to gather insights about the characteristics of the game's different class types

# Setup
***

In [1]:
# establishing environment
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Acquire
Acquiring data from local csv files
***

In [2]:
# reading in card data and saving as DF
cards = pd.read_csv('hearthstone_standard_cards.csv')

# reading in card classes and saving as DF
classes = pd.read_csv('classes.csv')

# reading in minion types data and saving as DF
mtypes = pd.read_csv('minionTypes.csv')

# reading in rarities data and saving as DF
rarities = pd.read_csv('rarities.csv')

# reading in set groups data and saving as DF
setgroups = pd.read_csv('setGroups.csv')

# reading in card sets data and saving as DF
sets = pd.read_csv('sets.csv')

# reading in types data and saving as DF
ctypes = pd.read_csv('types.csv')

# reading in keywords data and saving as DF
keywords = pd.read_csv('keywords.csv')

# Prepare
Preparing data for exploration
***

## Preparing DataFrames

### Lowercasing all DF columns and values

In [3]:
# creating list of all DFs
df_list = [cards, classes, mtypes, rarities, setgroups, sets, ctypes, keywords]

# iterating through DFs and lowercasing all column names and values
for dtafrm in df_list:
    dtafrm.columns = dtafrm.columns.str.lower()
    dtafrm = dtafrm.applymap(lambda s:s.lower() if type(s) == str else s)

## Merging DataFrames

### Merging 'classes' DF

In [4]:
# removing brackets and commas from multiclassids column
cards.multiclassids = cards.multiclassids.str.replace('\]|,|\[' , '')

# creating column to hold primary class id 
# if card is of one class, this will reflect its sole class
# if card is dual, this will reflect the 1st of the two classes in the multiClassIds column
# necessary since dual class cards erroneously hold the 'neutral' class value in their primary class id 
cards['primeclassid'] = np.where((cards.multiclassids.str.contains(' ')), cards["multiclassids"].str.split(" ", expand = True)[0], cards.classid)

# converting key columns to make all value data types match
cards.primeclassid = cards.primeclassid.astype(str)
classes.id = classes.id.astype(str)

# merging 'classes' df with card df
df = pd.merge(cards, classes[['id', 'slug']], 
              left_on = 'primeclassid', right_on = 'id', how="left", 
              suffixes = (None, '_prime_hero_class'))

# dropping columns I no longer need
df.drop(columns = ['primeclassid', 'classid'])

,id,collectible,slug,multiclassids,cardtypeid,cardsetid,rarityid,artistname,manacost,name,...,duels,miniontypeid,health,attack,keywordids,childids,durability,armor,id_prime_hero_class,slug_prime_hero_class
0,58607,1,58607-blur,,5,1463,1,Zoltan Boros,0,Blur,...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,demonhunter
1,56806,1,56806-shadowhoof-slayer,,4,2,2,A.J. Nazzaro,1,Shadowhoof Slayer,...,"{'relevant': True, 'constructed': True}",15.0,1.0,2.0,[8],NaN,NaN,NaN,14,demonhunter
2,58170,1,58170-crimson-sigil-runner,,4,1414,1,Arthur Bozonnet,1,Crimson Sigil Runner,...,NaN,NaN,1.0,1.0,[86],NaN,NaN,NaN,14,demonhunter
3,59394,1,59394-demon-companion,14 3,5,1443,3,Zoltan Boros,1,Demon Companion,...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,NaN,"[59391, 59392, 59393]",NaN,NaN,14,demonhunter
4,59606,1,59606-double-jump,,5,1443,1,A.J. Nazzaro,1,Double Jump,...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,[86],NaN,NaN,NaN,14,demonhunter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,61503,1,61503-cthun-the-shattered,,4,1466,5,Alex Horley Orlandelli,10,"C'Thun, the Shattered",...,"{'relevant': True, 'constructed': True}",NaN,6.0,6.0,"[8, 64]","[61873, 61875, 61874, 61877]",NaN,NaN,12,neutral
1285,61629,1,61629-darkmoon-rabbit,,4,1466,4,Matt Dixon,10,Darkmoon Rabbit,...,"{'relevant': True, 'constructed': True}",20.0,1.0,1.0,"[32, 53]",NaN,NaN,NaN,12,neutral
1286,60443,1,60443-nzoth-god-of-the-deep,,4,1466,5,Alex Horley Orlandelli,10,"N'Zoth, God of the Deep",...,"{'relevant': True, 'constructed': True}",NaN,7.0,5.0,[8],NaN,NaN,NaN,12,neutral
1287,61308,1,61308-yshaarj-the-defiler,,4,1466,5,Alex Horley Orlandelli,10,"Y'Shaarj, the Defiler",...,"{'relevant': True, 'constructed': True}",NaN,10.0,10.0,"[8, 98]",[61296],NaN,NaN,12,neutral


### Merging 'mtypes' DF

In [5]:
# changing null values of minionTypeId for neutral minions to -1
df['miniontypeid'] = np.where((df.miniontypeid.isnull() == True) & (df.cardtypeid == 4), -1, df.miniontypeid)

# adding missing keyword data to 'keywords' df
# -1 is for minions with no tribe
mtypes.loc[len(mtypes.index)] = ['no tribe', -1, 'No Tribe']

# merging 'mtypes' df
df = pd.merge(df, mtypes[['id', 'slug']], 
              left_on = 'miniontypeid', right_on = 'id', how="left", 
              suffixes = (None, '_minion_type'))

### Merging 'rarities' DF

In [6]:
# merging 'rarities' df
df = pd.merge(df, rarities[['id', 'slug']], 
              left_on = 'rarityid', right_on = 'id', how="left", 
              suffixes = (None, '_rarity'))

### Merging 'setGroups' DF

In [7]:
# merging 'setgroups' df
df = pd.merge(df, sets[['id', 'slug', 'name']], 
              left_on = 'cardsetid', right_on = 'id', how="left", 
              suffixes = (None, '_set'))

### Merging 'ctypes' DF

In [8]:
# merging 'ctypes' df
df = pd.merge(df, ctypes[['id', 'slug']], 
              left_on = 'cardtypeid', right_on = 'id', how="left", 
              suffixes = (None, '_card_type'))

### Merging 'keywords' DF

In [9]:
# adding missing keyword data to 'keywords' df
keywords.loc[len(keywords.index)] = ['64', 'start-of-game', 'start of game', 
                                     'does something at the start of the game.', 
                                     'does something at the start of the game.']

# removing brackets and commas from keyword id column
df.keywordids = df.keywordids.str.replace('\]|,|\[' , '')

# splitting keyword ids into separate columns for each card
kwdf = df["keywordids"].str.split(" ", expand = True) 

# renaming columns
kwdf.columns = ['keywordid1', 'keywordid2', 'keywordid3', 'keywordid4', 'keywordid5']

# concatenating split keyword id columns with main df
df = pd.concat([df, kwdf], axis=1)

# converting keywords id column to str type to enable merge
keywords.id = keywords.id.astype(str)

# creating loop to add a column for the text name of each keyword ability of each card
# via merging with keywords DF
for x in kwdf.columns:
    df = pd.merge(df, keywords[['id', 'slug']], 
              left_on = x, right_on = 'id', how = "left",
              suffixes = (None, x + '_name'))

## Checking for duplicate rows

In [10]:
# checking number of rows in current DF
df.shape

(1289, 51)

In [11]:
# Checking number of rows if duplicates were dropped
df.drop_duplicates().shape

(1289, 51)

- No duplicates found

### Checking for proper data types, categorical columns (based on domain knowledge), and null counts

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1289 entries, 0 to 1288
Data columns (total 51 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1289 non-null   int64  
 1   collectible            1289 non-null   int64  
 2   slug                   1289 non-null   object 
 3   classid                1289 non-null   int64  
 4   multiclassids          1289 non-null   object 
 5   cardtypeid             1289 non-null   int64  
 6   cardsetid              1289 non-null   int64  
 7   rarityid               1289 non-null   int64  
 8   artistname             1288 non-null   object 
 9   manacost               1289 non-null   int64  
 10  name                   1289 non-null   object 
 11  text                   1271 non-null   object 
 12  image                  1289 non-null   object 
 13  imagegold              805 non-null    object 
 14  flavortext             1289 non-null   object 
 15  crop

- The following columns will be dropped as they won't be needed for the expected operations of this project
    - id, slug
        - unique identifiers for cards, not needed since the 'name' column provides this while also being easier to reference
    - artistname, image, imagegold, cropimage
        - I won't be exploring images or artist names in this iteration of the project
    - all columns reflecting key words with the exception of the boolean columns and the 'slug_keyword#_name' columns
        - The exempted columns are sufficient for the project's expected operations
     
     
- Based on my domain knowledge of the game, I'm inferring that several of the columns are categorical
    - I need to create boolean columns for categorical columns (rarity, card set, etc.)


- Many null values that need to be addressed
    - text
    - duels
    - minion type id
    - health
    - attack
    - child ids
    - durability
    - armor

### Dropping columns that aren't needed for the planned operations of this project

In [13]:
# creating list of columns to drop
columns_to_drop = ['id', 'slug', 'artistname', 'image', 'imagegold', 'flavortext', 'cropimage']

# dropping columns
df.drop(columns = columns_to_drop, inplace = True)

# Addressing Null Values

### Addressing nulls in 'text' column

In [14]:
# checking values in text box
df.text.value_counts(dropna = False)

NaN                                                                                     18
<b>Taunt</b>                                                                            15
<b>Charge</b>                                                                            7
<b>Stealth</b>                                                                           6
<b>Spell Damage +1</b>                                                                   6
                                                                                        ..
<b>Choose One -</b> Give a minion +4 Attack; or +4 Health and <b>Taunt</b>.              1
<b>Lifesteal</b> <b>Deathrattle:</b> Shuffle 'Reliquary Prime' into your deck.           1
Add 2 random spells from another class to&nbsp;your hand.                                1
Whenever a friendly minion&nbsp;takes damage, gain 1 Armor.                              1
<b>Taunt</b> <b>Deathrattle:</b> Summon a 7/7 Felcracked Colossus with <b>Taunt</b>.     1

In [15]:
# filling null text values with 'no effect'
df["text"].fillna("no effect", inplace = True) 

### Addressing nulls in 'duels' column

In [16]:
# checking duels values
df.duels.value_counts(dropna = False)

{'relevant': True, 'constructed': True}    708
NaN                                        581
Name: duels, dtype: int64

In [17]:
# updating duels column so that cards that were allowed in duels have value of 1 and 0 otherwise
df['duels'] = np.where((df.duels == "{'relevant': True, 'constructed': True}"), 1, 0)

### Addressing nulls in 'minionTypeId', 'id_minion_type', and 'slug_minion_type' columns

In [18]:
# checking minontypeId values
df.miniontypeid.value_counts(dropna = False)

-1.0     497
 NaN     471
 20.0     75
 15.0     63
 18.0     51
 24.0     50
 17.0     32
 14.0     25
 23.0     18
 21.0      6
 26.0      1
Name: miniontypeid, dtype: int64

In [19]:
# converting nulls, aka non-minion cards to 'not a minion' type
df['id_minion_type'] = np.where((df.id_minion_type.isnull() == True), 'not a minion', df.id_minion_type)
df['slug_minion_type'] = np.where((df.slug_minion_type.isnull() == True), 'not a minion', df.slug_minion_type)

# dropping minionTypeId since id_minion_type suffices
df.drop(columns = 'miniontypeid', inplace = True)

### Addressing nulls in 'childIds' column

In [20]:
# checking childIds values
df.childids.value_counts(dropna = False)

NaN                                                                       994
[52897, 52900, 53160, 53161, 53162, 53163, 55378, 60588, 64652, 64653]     15
[59723]                                                                     8
[56927]                                                                     3
[53921]                                                                     3
                                                                         ... 
[600]                                                                       1
[54370]                                                                     1
[61449, 61450]                                                              1
[53241, 52875, 53244, 53246]                                                1
[60256]                                                                     1
Name: childids, Length: 261, dtype: int64

In [21]:
# filling nulls with "no_childid"
df.childids.fillna("no_childid", inplace = True) 

### Addressing nulls in 'health', 'attack', 'durability', and 'armor' columns
All of these variables respective columns have null values since none of these variables apply to every card (examples: only minions have health while only weapons have durability). For the time being I'll fill these nulls 
with a value that represents infinity. The benefit of this method is that it allows me to fill the nulls while preserving the int64 data type of the column. Furthermore, no matter what value blizzard assigns to these variables in future cards, this value probably wouldn't be used. If this causes issues later I'll employ a different means of handling them. 

In [22]:
# creating list of column names
hada = ['health', 'attack', 'durability', 'armor']

# iterating through columns filling nulls within each
for att in hada:
    df[att].fillna(float('inf'), inplace = True)

# Creating boolean columns for categorical variables

### Creating boolean columns for 'keywords'

In [23]:
# loop iterates through each keyword and creates a boolean column for it
for kw in keywords.slug:
    df['has_' + kw] = np.where(
    (df.slugkeywordid1_name == kw) |
    (df.slugkeywordid2_name == kw) |
    (df.slugkeywordid3_name == kw) |
    (df.slugkeywordid4_name == kw) |
    (df.slugkeywordid5_name == kw), 1, 0)
    
# creating empty list
key_word_col_drop = []

# iterating through columns in df and creating list of columns to drop
for col in df.columns:
    if 'keywordid' in col:
        key_word_col_drop.append(col)
        
# dropping columns
df.drop(columns = key_word_col_drop, inplace = True)

### Creating boolean columns for 'hero classes'

In [24]:
# removing brackets and commas from multiclassids column
df.multiclassids = df.multiclassids.str.replace('\]|,|\[' , '')

# creating column that holds secondary class separate from primary class
df['id_second_hero_class'] = df["multiclassids"].str.split(" ", expand = True)[1]

# converting column to str type to enable merge with newly created column 'classId2'
classes.id = classes.id.astype(str)

# creating df containing columns for merge in order to rename before merge without altering original classes DF
classes2 = classes[['id', 'slug']]

# renaming columns
classes2.columns = ['id_second_hero_class', 'slug_second_hero_class']

# merging 'classes' on secondary hero class id to get secondary class names
df = pd.merge(df, classes2[['id_second_hero_class', 'slug_second_hero_class']], 
              on = 'id_second_hero_class', how = "left")

# creating boolean columns for each hero class
for c in classes.slug:
    df['is_' + c] = np.where(
    (df.slug_prime_hero_class == c) | (df.slug_second_hero_class == c), 1, 0)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1289 entries, 0 to 1288
Data columns (total 78 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   collectible             1289 non-null   int64  
 1   classid                 1289 non-null   int64  
 2   multiclassids           1289 non-null   object 
 3   cardtypeid              1289 non-null   int64  
 4   cardsetid               1289 non-null   int64  
 5   rarityid                1289 non-null   int64  
 6   manacost                1289 non-null   int64  
 7   name                    1289 non-null   object 
 8   text                    1289 non-null   object 
 9   duels                   1289 non-null   int64  
 10  health                  1289 non-null   float64
 11  attack                  1289 non-null   float64
 12  childids                1289 non-null   object 
 13  durability              1289 non-null   float64
 14  armor                   1289 non-null   